## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(r"C:\Users\sherr\Desktop\Case_Data_Bootcamp\World_Weather_Analysis\Weather_Database\WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chuy,UY,2022-04-17 21:45:58,-33.6971,-53.4616,62.83,75,100,7.05,overcast clouds
1,1,Puerto Ayora,EC,2022-04-17 21:45:59,-0.7393,-90.3518,78.76,89,73,9.62,light rain
2,2,Tuktoyaktuk,CA,2022-04-17 21:45:59,69.4541,-133.0374,17.60,57,100,11.50,overcast clouds
3,3,Okakarara,NaN,2022-04-17 21:46:00,-20.5833,17.4333,70.86,58,84,4.23,broken clouds
4,4,Camacha,PT,2022-04-17 21:46:00,33.0833,-16.3333,60.22,77,40,9.22,scattered clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df=city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                210
City                   210
Country                210
Date                   210
Lat                    210
Lng                    210
Max Temp               210
Humidity               210
Cloudiness             210
Wind Speed             210
Current Description    210
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Puerto Ayora,EC,2022-04-17 21:45:59,-0.7393,-90.3518,78.76,89,73,9.62,light rain
5,5,Vaini,TO,2022-04-17 21:46:00,-21.2000,-175.2000,82.56,83,75,4.61,broken clouds
7,7,Rikitea,PF,2022-04-17 21:46:01,-23.1203,-134.9692,78.49,67,1,11.74,clear sky
14,14,Nioro,GM,2022-04-17 21:46:03,13.3500,-15.7500,79.92,44,4,10.67,clear sky
16,16,Georgetown,MY,2022-04-17 21:43:57,5.4112,100.3354,82.33,99,20,2.30,few clouds
17,17,Hithadhoo,MV,2022-04-17 21:46:04,-0.6000,73.0833,82.51,76,44,6.20,scattered clouds
35,35,Kapaa,US,2022-04-17 21:46:10,22.0752,-159.3190,78.78,89,75,17.27,light rain
41,41,Maracaibo,VE,2022-04-17 21:46:12,10.6317,-71.6406,82.56,69,95,8.32,overcast clouds
45,45,Grajau,BR,2022-04-17 21:47:13,-5.8194,-46.1386,75.72,93,98,2.04,overcast clouds
51,51,Acapulco,MX,2022-04-17 21:38:19,16.8634,-99.8901,85.82,66,0,11.50,clear sky


In [17]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,
5,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
7,Rikitea,PF,78.49,clear sky,-23.1203,-134.9692,
14,Nioro,GM,79.92,clear sky,13.3500,-15.7500,
16,Georgetown,MY,82.33,few clouds,5.4112,100.3354,
17,Hithadhoo,MV,82.51,scattered clouds,-0.6000,73.0833,
35,Kapaa,US,78.78,light rain,22.0752,-159.3190,
41,Maracaibo,VE,82.56,overcast clouds,10.6317,-71.6406,
45,Grajau,BR,75.72,overcast clouds,-5.8194,-46.1386,
51,Acapulco,MX,85.82,clear sky,16.8634,-99.8901,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, NameError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df = clean_hotel_df[clean_hotel_df["Hotel Name"] != ""]
clean_hotel_df.head(30)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
5,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
7,Rikitea,PF,78.49,clear sky,-23.1203,-134.9692,People ThankYou
16,Georgetown,MY,82.33,few clouds,5.4112,100.3354,Cititel Penang
17,Hithadhoo,MV,82.51,scattered clouds,-0.6000,73.0833,Scoop Guest House
35,Kapaa,US,78.78,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
41,Maracaibo,VE,82.56,overcast clouds,10.6317,-71.6406,Motel Aladdin
45,Grajau,BR,75.72,overcast clouds,-5.8194,-46.1386,Hotel Confiança
51,Acapulco,MX,85.82,clear sky,16.8634,-99.8901,HS Hotsson Smart Acapulco
56,Lorengau,PG,82.49,scattered clouds,-2.0226,147.2712,Lorengau Harbourside Hotel


In [28]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}<dd>
<dt>City</dt><dd>{City}<dd/>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}<dd>
</d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [37]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure(center=(30,31),zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))